<a href="https://colab.research.google.com/github/juanfisicobr/Red-coocurrencia-terminos/blob/main/ClusterTem%C3%A1ticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas nltk python-louvain matplotlib networkx --force-reinstall

In [19]:
import pandas as pd
import networkx as nx
import re
from itertools import combinations
import community as community_louvain
import matplotlib.pyplot as plt
import unicodedata
import numpy as np

def _eliminar_tildes(texto):
    nfkd_form = unicodedata.normalize('NFD', texto)
    return "".join([c for c in nfkd_form if unicodedata.category(c) != 'Mn'])

def preprocess_text(text, custom_stopwords=None):
    mapa_normalizacao = {
        'investigaciones': 'investigacion',
        'docentes': 'docente',
        'estudiantes': 'estudiante',
        'sociales': 'social',
        'educacionales': 'educacional',
        'nacionales': 'nacional',
        'regionales': 'regional',
        'locales': 'local',
        'institucionales': 'institucional',
        'profesionales': 'profesional',
        'populares': 'popular',
        'municipales': 'municipal',
        'políticas': 'política',
        'acciones': 'accion',
        'redes': 'red',
        'universidades': 'universidad'
    }
    stop_words = set(['de', 'a', 'o', 'que', 'y', 'e', 'el', 'la', 'en', 'un', 'una', 'para',
        'con', 'no', 'los', 'las', 'por', 'mas', 'más', 'se', 'su', 'sus',
        'como', 'pero', 'al', 'del', 'le', 'lo', 'me', 'mi', 'sin', 'son',
        'tambien', 'también', 'este', 'esta', 'estos', 'estas', 'ser', 'es'])
    if custom_stopwords:
        stop_words.update(custom_stopwords)
    text = text.lower()
    text = _eliminar_tildes(text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    tokens = text.split()
    normalized_tokens = [mapa_normalizacao.get(token, token) for token in tokens]
    filtered_tokens = [word for word in normalized_tokens if word not in stop_words and len(word) > 2]
    return filtered_tokens

def create_cooccurrence_matrix_from_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read()
    documents_raw = content.split('###')
    documents_raw = [doc.strip() for doc in documents_raw if doc.strip()]
    custom_stopwords = ['programa', 'educacion', 'pdi', 'art', 'articulo', 'sobre']
    processed_docs = [preprocess_text(doc, custom_stopwords) for doc in documents_raw]
    vocabulary = sorted(list(set(term for doc in processed_docs for term in doc)))
    M = pd.DataFrame(0, index=vocabulary, columns=vocabulary)
    for doc in processed_docs:
        unique_terms_in_doc = sorted(list(set(doc)))
        for term in unique_terms_in_doc:
            M.loc[term, term] += 1
        for term1, term2 in combinations(unique_terms_in_doc, 2):
            M.loc[term1, term2] += 1
            M.loc[term2, term1] += 1
    return M

def calcular_e_associar_metricas(G, M):
    partition = community_louvain.best_partition(G, weight='weight')
    pagerank = nx.pagerank(G, weight='weight')
    occurrences = {term: M.loc[term, term] for term in G.nodes()}
    clusters_ajustados = {node: cluster_id + 1 for node, cluster_id in partition.items()}
    nx.set_node_attributes(G, clusters_ajustados, 'cluster')
    nx.set_node_attributes(G, pagerank, 'pagerank')
    nx.set_node_attributes(G, occurrences, 'occurrences')
    print("Métricas (Cluster, PageRank, Ocorrencias) calculadas y asociadas a los nodos.")
    return G

def filtrar_rede(G, top_n, min_edge_weight_for_viz):
    if G.number_of_nodes() <= top_n:
        top_nodes = list(G.nodes())
    else:
        pagerank_dict = nx.get_node_attributes(G, 'pagerank')
        sorted_nodes = sorted(pagerank_dict, key=pagerank_dict.get, reverse=True)
        top_nodes = sorted_nodes[:top_n]
    G_sub = G.subgraph(top_nodes).copy()
    G_final = nx.Graph()
    G_final.add_nodes_from(G_sub.nodes(data=True))
    for u, v, data in G_sub.edges(data=True):
        if data['weight'] >= min_edge_weight_for_viz:
            G_final.add_edge(u, v, weight=data['weight'])
    G_final.remove_nodes_from(list(nx.isolates(G_final)))
    print(f"Red final (Top {top_n} nodos, Bordes >= {min_edge_weight_for_viz}): {G_final.number_of_nodes()} nodos, {G_final.number_of_edges()} bordes.")
    return G_final

## ----------------------------------------------------------------
## FUNCIÓN: Visualización de la Red
## ----------------------------------------------------------------
def visualizar_rede(G, title, output_filename):
    if G.number_of_nodes() == 0:
        print("La red está vacía. No es posible generar el gráfico.")
        return

    plt.figure(figsize=(16, 9))
    pos = nx.circular_layout(G)
    # Colores monocromáticos basados ​​en PageRank
    pagerank_values = [data.get('pagerank', 0) for _, data in G.nodes(data=True)]
    if pagerank_values:
        min_pr = min(pagerank_values)
        max_pr = max(pagerank_values)
        if max_pr == min_pr:
            norm_pr_values = [0.5] * len(pagerank_values)
        else:
            norm_pr_values = [(p - min_pr) / (max_pr - min_pr) for p in pagerank_values]
    else:
        norm_pr_values = []

    cmap = plt.cm.get_cmap('Blues_r')
    node_colors = [cmap(p) for p in norm_pr_values]

    # Tamaño del nodo
    min_size = 1500
    max_size = 16000
    if pagerank_values and min_pr == max_pr:
        node_sizes = [min_size] * G.number_of_nodes()
    elif pagerank_values:
        node_sizes = [min_size + ((p - min_pr) / (max_pr - min_pr)) * (max_size - min_size) for p in pagerank_values]
    else:
        node_sizes = []

    custom_labels = {
        node: f"{node.capitalize()}\n({data.get('occurrences', '?')})"
        for node, data in G.nodes(data=True)
    }


    nx.draw_networkx_nodes(
        G,
        pos,
        node_color=node_colors,
        node_size=node_sizes,
        alpha=0.7,
        edgecolors='black',
        linewidths=1.5
    )

    nx.draw_networkx_edges(G, pos, alpha=0.3, width=1.5, edge_color='grey')
    nx.draw_networkx_labels(G, pos, labels=custom_labels, font_size=12, font_color='black', font_weight='bold')

    plt.title(title, size=20)
    plt.tight_layout()
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"\nGráfico '{output_filename}' guardado con éxito!")

## ----------------------------------------------------------------
## BLOQUE DE EJECUCIÓN PRINCIPAL
## ----------------------------------------------------------------
if __name__ == "__main__":

    # --- Parámetros de Entrada ---
    FILE_PATH = 'extractos.txt'
    TOP_N_NODES = 20
    MIN_EDGE_WEIGHT_VIZ = 5 # Ajuste para controlar la densidad de la línea.

    GRAFICO_TITULO = "Análisis de Red Monocromática"
    GRAFICO_OUTPUT_FILE = "analisis_red_monocromatica.png"

    # --- Ejecución del flujo de trabajo ---
    matriz_M = create_cooccurrence_matrix_from_file(FILE_PATH)
    grafo_base = nx.from_pandas_adjacency(matriz_M)
    grafo_com_metricas = calcular_e_associar_metricas(grafo_base, matriz_M)
    grafo_final = filtrar_rede(grafo_com_metricas, top_n=TOP_N_NODES, min_edge_weight_for_viz=MIN_EDGE_WEIGHT_VIZ)
    visualizar_rede(grafo_final, GRAFICO_TITULO, GRAFICO_OUTPUT_FILE)

Métricas (Cluster, PageRank, Ocorrencias) calculadas y asociadas a los nodos.
Red final (Top 20 nodos, Bordes >= 5): 20 nodos, 39 bordes.


/tmp/ipython-input-2503961863.py:116: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('Blues_r')



Gráfico 'analisis_red_monocromatica.png' guardado con éxito!


In [16]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

## ----------------------------------------------------------------
## FUNCIÓN: Visualización de la Red (Modificada para Clusters)
## ----------------------------------------------------------------
def visualizar_rede(G, title, output_filename):
    if G.number_of_nodes() == 0:
        print("La red está vacía. No es posible generar el gráfico.")
        return

    plt.figure(figsize=(16, 9))
    pos = nx.circular_layout(G)


    cluster_values = [data.get('cluster', 0) for _, data in G.nodes(data=True)]
    try:
        cmap = plt.colormaps.get_cmap('tab10')
    except AttributeError:
        cmap = plt.cm.get_cmap('tab10')

    pagerank_values = [data.get('pagerank', 0) for _, data in G.nodes(data=True)]
    min_size = 1500
    max_size = 16000

    node_sizes = []
    if pagerank_values:
        min_pr = min(pagerank_values)
        max_pr = max(pagerank_values)
        if max_pr == min_pr:
            node_sizes = [min_size] * G.number_of_nodes()
        else:
            node_sizes = [
                min_size + ((p - min_pr) / (max_pr - min_pr)) * (max_size - min_size)
                for p in pagerank_values
            ]
    else:
        node_sizes = [min_size] * G.number_of_nodes()

    custom_labels = {
        node: f"{node.capitalize()}\n({data.get('occurrences', '?')})"
        for node, data in G.nodes(data=True)
    }

    # Extraer todos los pesos (weights) de las aristas
    edge_weights = [data['weight'] for u, v, data in G.edges(data=True)]

    base_width = 1.0          # El grosor de la línea más delgada
    max_extra_width = 6.0     # Cuánto se añade a la línea más gruesa

    scaled_widths = []
    if edge_weights:
        min_w = min(edge_weights)
        max_w = max(edge_weights)

        if max_w == min_w:
            scaled_widths = [base_width] * len(edge_weights)
        else:
            for w in edge_weights:
                norm_w = (w - min_w) / (max_w - min_w)
                scaled_widths.append(base_width + norm_w * max_extra_width)

    nx.draw_networkx_nodes(
        G,
        pos,
        node_color=cluster_values,
        cmap=cmap,
        node_size=node_sizes,
        alpha=0.8,
        edgecolors='black',
        linewidths=1.5
    )

    nx.draw_networkx_edges(
        G,
        pos,
        alpha=0.3,
        width=scaled_widths,
        edge_color='grey'
    )

    nx.draw_networkx_labels(G, pos, labels=custom_labels, font_size=12, font_color='black', font_weight='bold')

    plt.title(title, size=20)
    plt.tight_layout()
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"\nGráfico '{output_filename}' guardado con éxito! (Grosor de línea dinámico)")

## ----------------------------------------------------------------
## BLOQUE DE EJECUCIÓN PRINCIPAL
## ----------------------------------------------------------------
if __name__ == "__main__":

    # --- Parámetros de Entrada ---
    FILE_PATH = 'extractos.txt'
    TOP_N_NODES = 20
    MIN_EDGE_WEIGHT_VIZ = 5 # Ajuste para controlar la densidad de la línea.

    GRAFICO_TITULO = "Análisis de Red "
    GRAFICO_OUTPUT_FILE = "analisis_red_colorida.png"

    # --- Ejecución del flujo de trabajo ---
    matriz_M = create_cooccurrence_matrix_from_file(FILE_PATH)
    grafo_base = nx.from_pandas_adjacency(matriz_M)
    grafo_com_metricas = calcular_e_associar_metricas(grafo_base, matriz_M)
    grafo_final = filtrar_rede(grafo_com_metricas, top_n=TOP_N_NODES, min_edge_weight_for_viz=MIN_EDGE_WEIGHT_VIZ)
    visualizar_rede(grafo_final, GRAFICO_TITULO, GRAFICO_OUTPUT_FILE)

Métricas (Cluster, PageRank, Ocorrencias) calculadas y asociadas a los nodos.
Red final (Top 20 nodos, Bordes >= 5): 20 nodos, 40 bordes.

Gráfico 'analisis_red_colorida.png' guardado con éxito! (Grosor de línea dinámico)


In [20]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

## ----------------------------------------------------------------
## FUNCIÓN: Visualización de la Red (Modificada para Clusters)
## ----------------------------------------------------------------
def visualizar_rede(G, title, output_filename):
    if G.number_of_nodes() == 0:
        print("La red está vacía. No es posible generar el gráfico.")
        return

    plt.figure(figsize=(16, 9))
    pos = nx.kamada_kawai_layout(G)


    cluster_values = [data.get('cluster', 0) for _, data in G.nodes(data=True)]
    try:
        cmap = plt.colormaps.get_cmap('tab10')
    except AttributeError:
        cmap = plt.cm.get_cmap('tab10')

    pagerank_values = [data.get('pagerank', 0) for _, data in G.nodes(data=True)]
    min_size = 1500
    max_size = 16000

    node_sizes = []
    if pagerank_values:
        min_pr = min(pagerank_values)
        max_pr = max(pagerank_values)
        if max_pr == min_pr:
            node_sizes = [min_size] * G.number_of_nodes()
        else:
            node_sizes = [
                min_size + ((p - min_pr) / (max_pr - min_pr)) * (max_size - min_size)
                for p in pagerank_values
            ]
    else:
        node_sizes = [min_size] * G.number_of_nodes()

    custom_labels = {
        node: f"{node.capitalize()}\n({data.get('occurrences', '?')})"
        for node, data in G.nodes(data=True)
    }

    # Extraer todos los pesos (weights) de las aristas
    edge_weights = [data['weight'] for u, v, data in G.edges(data=True)]

    base_width = 1.0          # El grosor de la línea más delgada
    max_extra_width = 6.0     # Cuánto se añade a la línea más gruesa

    scaled_widths = []
    if edge_weights:
        min_w = min(edge_weights)
        max_w = max(edge_weights)

        if max_w == min_w:
            scaled_widths = [base_width] * len(edge_weights)
        else:
            for w in edge_weights:
                norm_w = (w - min_w) / (max_w - min_w)
                scaled_widths.append(base_width + norm_w * max_extra_width)

    nx.draw_networkx_nodes(
        G,
        pos,
        node_color=cluster_values,
        cmap=cmap,
        node_size=node_sizes,
        alpha=0.8,
        edgecolors='black',
        linewidths=1.5
    )

    nx.draw_networkx_edges(
        G,
        pos,
        alpha=0.3,
        width=scaled_widths,
        edge_color='grey'
    )

    nx.draw_networkx_labels(G, pos, labels=custom_labels, font_size=12, font_color='black', font_weight='bold')

    plt.title(title, size=20)
    plt.tight_layout()
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"\nGráfico '{output_filename}' guardado con éxito! (Grosor de línea dinámico)")

## ----------------------------------------------------------------
## BLOQUE DE EJECUCIÓN PRINCIPAL
## ----------------------------------------------------------------
if __name__ == "__main__":

    # --- Parámetros de Entrada ---
    FILE_PATH = 'extractos.txt'
    TOP_N_NODES = 20
    MIN_EDGE_WEIGHT_VIZ = 4 # Ajuste para controlar la densidad de la línea.

    GRAFICO_TITULO = "Análisis de Red"
    GRAFICO_OUTPUT_FILE = "analisis_red_colorida_Kamada.png"

    # --- Ejecución del flujo de trabajo ---
    matriz_M = create_cooccurrence_matrix_from_file(FILE_PATH)
    grafo_base = nx.from_pandas_adjacency(matriz_M)
    grafo_com_metricas = calcular_e_associar_metricas(grafo_base, matriz_M)
    grafo_final = filtrar_rede(grafo_com_metricas, top_n=TOP_N_NODES, min_edge_weight_for_viz=MIN_EDGE_WEIGHT_VIZ)
    visualizar_rede(grafo_final, GRAFICO_TITULO, GRAFICO_OUTPUT_FILE)

Métricas (Cluster, PageRank, Ocorrencias) calculadas y asociadas a los nodos.
Red final (Top 20 nodos, Bordes >= 4): 20 nodos, 54 bordes.

Gráfico 'analisis_red_colorida_Kamada.png' guardado con éxito! (Grosor de línea dinámico)


In [22]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

## ----------------------------------------------------------------
## FUNCIÓN: Visualización de la Red (Monocromática + Grosor Dinámico)
## ----------------------------------------------------------------
def visualizar_rede(G, title, output_filename):

    if G.number_of_nodes() == 0:
        print("La red está vacía. No es posible generar el gráfico.")
        return

    plt.figure(figsize=(16, 9))

    pos = nx.kamada_kawai_layout(G)

    # --- Lógica de Tamaño y Color (PageRank) ---
    pagerank_values = [data.get('pagerank', 0) for _, data in G.nodes(data=True)]

    # --- Lógica de Tamaño ---
    min_size = 1500
    max_size = 16000
    node_sizes = []
    if pagerank_values:
        min_pr = min(pagerank_values)
        max_pr = max(pagerank_values)
        if max_pr == min_pr:
            node_sizes = [min_size] * G.number_of_nodes()
        else:
            node_sizes = [
                min_size + ((p - min_pr) / (max_pr - min_pr)) * (max_size - min_size)
                for p in pagerank_values
            ]
    else:
        node_sizes = [min_size] * G.number_of_nodes()


    try:
        cmap_color = plt.colormaps.get_cmap('Blues_r')
    except AttributeError:
        # Para versiones antiguas de Matplotlib
        cmap_color = plt.cm.get_cmap('Blues_r')

    # --- (MANTENIDO) Lógica de Grosor de Línea (Peso de Arista) ---
    edge_weights = [data['weight'] for u, v, data in G.edges(data=True)]
    base_width = 1.0
    max_extra_width = 6.0
    scaled_widths = []
    if edge_weights:
        min_w = min(edge_weights)
        max_w = max(edge_weights)
        if max_w == min_w:
            scaled_widths = [base_width] * len(edge_weights)
        else:
            scaled_widths = [
                base_width + (((w - min_w) / (max_w - min_w)) * max_extra_width)
                for w in edge_weights
            ]
    else:
        scaled_widths = [base_width] * len(edge_weights)

    custom_labels = {
        node: f"{node.capitalize()}\n({data.get('occurrences', '?')})"
        for node, data in G.nodes(data=True)
    }

    # --- Dibujar Nodos ---
    nx.draw_networkx_nodes(
        G,
        pos,
        node_color=pagerank_values,
        cmap=cmap_color,
        # ---
        node_size=node_sizes,
        alpha=0.8,
        edgecolors='black',
        linewidths=1.5
    )

    # --- Dibujar Aristas ---
    nx.draw_networkx_edges(
        G,
        pos,
        alpha=0.3,
        width=scaled_widths,  # <-- Se mantiene el grosor dinámico
        edge_color='grey'
    )

    # --- Dibujar Etiquetas ---
    nx.draw_networkx_labels(G, pos, labels=custom_labels, font_size=12, font_color='black', font_weight='bold')

    plt.title(title, size=20)
    plt.tight_layout()
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"\nGráfico '{output_filename}' guardado con éxito! (Monocromático y grosor dinámico)")
## ----------------------------------------------------------------
## BLOQUE DE EJECUCIÓN PRINCIPAL
## ----------------------------------------------------------------
if __name__ == "__main__":

    # --- Parámetros de Entrada ---
    FILE_PATH = 'extractos.txt'
    TOP_N_NODES = 20
    MIN_EDGE_WEIGHT_VIZ = 5 # Ajuste para controlar la densidad de la línea.

    GRAFICO_TITULO = "Análisis de Red"
    GRAFICO_OUTPUT_FILE = "analisis_red_monocromática_Kamada.png"

    # --- Ejecución del flujo de trabajo ---
    matriz_M = create_cooccurrence_matrix_from_file(FILE_PATH)
    grafo_base = nx.from_pandas_adjacency(matriz_M)
    grafo_com_metricas = calcular_e_associar_metricas(grafo_base, matriz_M)
    grafo_final = filtrar_rede(grafo_com_metricas, top_n=TOP_N_NODES, min_edge_weight_for_viz=MIN_EDGE_WEIGHT_VIZ)
    visualizar_rede(grafo_final, GRAFICO_TITULO, GRAFICO_OUTPUT_FILE)

Métricas (Cluster, PageRank, Ocorrencias) calculadas y asociadas a los nodos.
Red final (Top 20 nodos, Bordes >= 5): 20 nodos, 39 bordes.

Gráfico 'analisis_red_monocromática_Kamada.png' guardado con éxito! (Monocromático y grosor dinámico)


In [17]:
## ----------------------------------------------------------------
## BLOQUE DE EJECUCIÓN PRINCIPAL (CON PASO DE EDICIÓN)
## ----------------------------------------------------------------
if __name__ == "__main__":

    # --- Parámetros de Entrada ---
    FILE_PATH = 'extractos.txt'
    TOP_N_NODES = 20
    MIN_EDGE_WEIGHT_VIZ = 5
    GRAFICO_TITULO = "Análisis de Red (Clusters y Correlación)"
    GRAFICO_OUTPUT_FILE = "analisis_red_clusters.png"

    # Nombre del archivo para editar las etiquetas
    MAPEO_ETIQUETAS_FILE = 'mapeo_terminos.csv'

    # --- 1. Flujo de trabajo estándar (hasta el filtrado) ---
    print("Iniciando análisis...")
    matriz_M = create_cooccurrence_matrix_from_file(FILE_PATH)
    grafo_base = nx.from_pandas_adjacency(matriz_M)
    grafo_com_metricas = calcular_e_associar_metricas(grafo_base, matriz_M)
    grafo_final = filtrar_rede(grafo_com_metricas, top_n=TOP_N_NODES, min_edge_weight_for_viz=MIN_EDGE_WEIGHT_VIZ)

    # --- 2. (NUEVO) Exportar nodos para corrección de tildes ---

    # Obtener la lista de nodos del grafo final
    nodos_sin_tildes = list(grafo_final.nodes())

    # Crear un DataFrame de Pandas: original -> corregido
    # Inicialmente, ambas columnas son iguales.
    df_mapa = pd.DataFrame({
        'original_sin_tilde': nodos_sin_tildes,
        'corregido_con_tilde': nodos_sin_tildes
    })

    # Guardar en un archivo CSV
    df_mapa.to_csv(MAPEO_ETIQUETAS_FILE, index=False, encoding='utf-8-sig')

    # --- 3. (NUEVO) Pausa para la edición manual ---
    print("-" * 70)
    print(f"ARCHIVO CREADO: '{MAPEO_ETIQUETAS_FILE}'")
    print("Por favor, abre este archivo CSV (con Excel, Google Sheets, o un editor de texto).")
    print("Modifica la columna 'corregido_con_tilde' para añadir las tildes necesarias.")
    print("-" * 70)

    # Pausa el script y espera a que el usuario presione Enter
    input(">>> PRESIONA ENTER para continuar después de guardar tus cambios... ")

    # --- 4. (NUEVO) Importar mapa corregido y re-etiquetar ---
    print("Leyendo el archivo de etiquetas corregido...")

    # Leer el archivo que acabas de editar
    try:
        df_mapa_editado = pd.read_csv(MAPEO_ETIQUETAS_FILE, encoding='utf-8-sig')
    except Exception as e:
        print(f"Error leyendo el archivo {MAPEO_ETIQUETAS_FILE}: {e}")
        print("Asegúrate de que el archivo esté guardado correctamente.")
        exit()

    # Crear el diccionario de mapeo
    # (ej. {'investigacion': 'investigación', 'docente': 'docente', ...})
    mapa_de_etiquetas = pd.Series(
        df_mapa_editado.corregido_con_tilde.values,
        index=df_mapa_editado.original_sin_tilde
    ).to_dict()

    # Aplicar el re-etiquetado al grafo
    grafo_etiquetado = nx.relabel_nodes(grafo_final, mapa_de_etiquetas, copy=True)

    print("Nodos re-etiquetados con éxito.")

    # --- 5. Ejecución del flujo de trabajo (Visualización) ---
    visualizar_rede(
        grafo_etiquetado,  # <--- Usamos el nuevo grafo con tildes
        GRAFICO_TITULO,
        GRAFICO_OUTPUT_FILE
    )

Iniciando análisis...
Métricas (Cluster, PageRank, Ocorrencias) calculadas y asociadas a los nodos.
Red final (Top 20 nodos, Bordes >= 5): 20 nodos, 40 bordes.
----------------------------------------------------------------------
ARCHIVO CREADO: 'mapeo_terminos.csv'
Por favor, abre este archivo CSV (con Excel, Google Sheets, o un editor de texto).
Modifica la columna 'corregido_con_tilde' para añadir las tildes necesarias.
----------------------------------------------------------------------
>>> PRESIONA ENTER para continuar después de guardar tus cambios... 
Leyendo el archivo de etiquetas corregido...
Nodos re-etiquetados con éxito.

Gráfico 'analisis_red_clusters.png' guardado con éxito! (Grosor de línea dinámico)
